* stopwords
* cross validation na hiperparametrach
* 

# Model description



---



---



*The aim of this model is not to achieve high accuracy but to present way of 
solving mentioned problem. Accuracy probably will be low due to small amount of articles for training and it's labels.*



---



Below model ought to predict whether the article is about money laundering activity or not. Then if it is model will predict whether the article is in tone of allegations or convictions.

To make it work I will use 2 models trained on 2 separate datasets.
* 1 dataset is assembled of 25 articles about money laundering and other subjects articles. Thats how model will learn how to distinguish money laundering article from the others. All variables' names belonging to this model start from 'ml' from Money Laundering.
* 2 dataset is assembled of 20 articles with allegations or convictions. Thats how model will learn how to distinguish convictions from allegations. All variables' names belonging to this model start from 'ind' from Indictment.

After creating two separate models I will use them accordingly on the new articles df assembled from articles without labels. Thus, ml_model will be launched on the articles df first to filter out money activity articles only, creating ml_model df. Subsequently ind_model will separate allegations articles and conviction articles from ml_mode df, creating final ind_model df. All can be run through Application.

# Libraries



---



---



In [60]:
import pandas as pd
import numpy as np
# np.random.seed(1337)

from tensorflow.keras.preprocessing.text import text_to_word_sequence as tws
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SimpleRNN, Bidirectional

# Data import



---



---



In [61]:
# training dataset for money laundering news or not classification
df_ml = pd.read_excel('train.xlsx', sheet_name = 'money_laundering')

# training dataset for conviction or not classification
df_ind = pd.read_excel('train.xlsx', sheet_name = 'allegations_convictions')

# Exploratory Data Analysis



---



---



Since I've created this training data and it's quite small EDA look's a bit different than regular one



---



First part of task is ambigious. For the sake of the project I treated "money laundering activity" as an article describing money laundering offence. So, the article about "How bank XYZ want's to improve their money laundering security" or "How cryptocurrency can be used as money laundering tool" are marked as non-money laundering at training set because they are not describing criminal offence. On the contrary article named "Mr. Whitecollar accused of money laundering" would be marked as money laundering.

## money laundering df

* 1 - money laundering activity
* 0 - not a money laundering activity

df_ml training set incudes different topics articles

In [62]:
print('Allegation or Conviction training dataset:\n{}'.format(df_ml.head(5)))
print('\n')
print('Length: \n{} rows'.format(len(df_ml)))
print('\n')
print('Example: \n{} {}\n\n{}'.format('Class:','Not Money Laundering' if df_ml.iloc[0,1] == 0 else 'Money Laundering',df_ml.iloc[0,0]))

Allegation or Conviction training dataset:
                                                text  ml
0  Dublin criminal Graham Whelan, who used crime ...   1
1  Man jailed for pushing woman from Dunkeld Brid...   0
2  Many people were surprised to learn that Ninte...   0
3  New York’s largest health care provider fires ...   0
4  Nobel in physics: Climate science breakthrough...   0


Length: 
25 rows


Example: 
Class: Money Laundering

Dublin criminal Graham Whelan, who used crime cash to pay for a penthouse suite at a luxury Dublin hotel and to refurbish his home, has made a “dramatic change” to his life since the tragic death of his wife, the Special Criminal Court has heard.

Whelan (39) of Walkinstown Avenue, Walkinstown, Dublin 12, pleaded guilty in July at the Special Criminal Court to participating in the actions of an organised crime group by laundering money for the group and being in possession of €1,200 in cash and a €28,000 watch, the proceeds of crime at the Intercontinen

## indictment df

* 1 - conviction case
* 0 - allegation case

df_ind training set includes money laundering activity articles only

In [63]:
print('Allegation or Conviction training dataset:\n{}'.format(df_ind.head(5)))
print('\n')
print('Length: \n{} rows'.format(len(df_ind)))
print('\n')
print('Example: \n{} {}\n\n{}'.format('Class:','Allegtion' if df_ind.iloc[0,1] == 0 else 'Conviction',df_ind.iloc[0,0]))

Allegation or Conviction training dataset:
                                                text  conviction
0  Alleged Money Laundering: Court fixes 14 Octob...           0
1  Pandora Papers Reveal Artworld Money Launderin...           0
2  Ex-adviser jailed after laundering money for o...           1
3  Comancheros gang president Pasilika Naufahu ha...           1
4  Australia: More people charged with money laun...           0


Length: 
20 rows


Example: 
Class: Allegtion

Alleged Money Laundering: Court fixes 14 October for ex-PDP’s spokesperson, Olisa Metuh’s retrial
Earlier jailed for the money laundering offences, Olisa Metuh was released from prison in December 2020 after the Court of Appeal set aside his earlier conviction and sentence.
ByPremium TimesandAgency Report  September 28, 2021 3 min read
The Federal High Court in Abuja on Tuesday fixed 14 October for re-arraignment of a formerly convicted former Peoples Democratic Party (PDP)’s spokesperson, Olisa Metuh.

This sign

# Feature Engineering 



---



---



Creating list of texts for each dataset

## money laundering txt and class lists

In [64]:
ml_txt = []
ml_class = []
for i in df_ml['text']:
  ml_txt.append(i)
for i in df_ml['ml']:
  ml_class.append(i)

In [65]:
ml_txt[:3]

['Dublin criminal Graham Whelan, who used crime cash to pay for a penthouse suite at a luxury Dublin hotel and to refurbish his home, has made a “dramatic change” to his life since the tragic death of his wife, the Special Criminal Court has heard.\n\nWhelan (39) of Walkinstown Avenue, Walkinstown, Dublin 12, pleaded guilty in July at the Special Criminal Court to participating in the actions of an organised crime group by laundering money for the group and being in possession of €1,200 in cash and a €28,000 watch, the proceeds of crime at the Intercontinental Hotel, Ballsbridge, Dublin 4, on January 31th, 2019.\n\nWhelan also pleaded guilty to paying €2,140 for a three-night stay at the hotel at Room 342, knowing or believing that the money was the proceeds of criminal conduct, contrary to Section 7 of the Act.\n\nWhen gardaí asked Whelan on the night how he could account for the €1,275 in cash he had in his pocket, he told officers that he had gotten it from “up his Swiss roll” and t

In [66]:
ml_class[:3]

[1, 0, 0]

## indictment txt and class lists

In [67]:
ind_txt = []
ind_class = []
for i in df_ind['text']:
  ind_txt.append(i)
for i in df_ind['conviction']:
  ind_class.append(i)

In [68]:
ind_txt[:3]

['Alleged Money Laundering: Court fixes 14 October for ex-PDP’s spokesperson, Olisa Metuh’s retrial\nEarlier jailed for the money laundering offences, Olisa Metuh was released from prison in December 2020 after the Court of Appeal set aside his earlier conviction and sentence.\nByPremium TimesandAgency Report  September 28, 2021 3 min read\nThe Federal High Court in Abuja on Tuesday fixed 14 October for re-arraignment of a formerly convicted former Peoples Democratic Party (PDP)’s spokesperson, Olisa Metuh.\n\nThis signals the commencement of his retrial on money laundering charges as ordered by the Court of Appeal in December 2020.\n\nMr Metuh faces charges of laundering N400million he allegedly received fraudulently from a former National Security Adviser, Sambo Dasuki, for his party’s presidential campaigns ahead of the 2015 elections.\n\nTwo of the charges have to do with $2million cash he allegedly transacted with without going through a financial institution, in violation of cash

In [69]:
ind_class[:3]

[0, 0, 1]

# Money Laundering activity detection



---



---



### Preprocessing hyperparameters

In [70]:
words_cnt = 1000 # Tokenizing - most common words

max_len = 200    # Pad sequences - each article has different number of words/sequences to make it even for all articles we will
                  # use pad_sequences method reducing/extending each up to 1000 words

target_dim = 10   # Embedding - we want to reduce dimensionality to 50

## Stopwords

In [71]:
from gensim.parsing.preprocessing import remove_stopwords

for i in range(len(ml_txt)):
  ml_txt[i] = remove_stopwords(ml_txt[i])

## Tokenizing

In [72]:
ml_tokenizer = Tokenizer(num_words = words_cnt)
ml_tokenizer.fit_on_texts(ml_txt)

In [73]:
list(ml_tokenizer.index_word.items())[:20]

[(1, 'the'),
 (2, 'money'),
 (3, 'said'),
 (4, 'laundering'),
 (5, 'climate'),
 (6, 'crypto'),
 (7, 'cryptocurrency'),
 (8, 'new'),
 (9, '”'),
 (10, 'people'),
 (11, 'tax'),
 (12, 'it'),
 (13, 'mr'),
 (14, 'in'),
 (15, 'court'),
 (16, 'property'),
 (17, 'change'),
 (18, 'year'),
 (19, 'funds'),
 (20, 'goxx')]

## Sequencing

In [74]:
# Sequencing - change each word in the article to ascribed token

ml_sequence = ml_tokenizer.texts_to_sequences(ml_txt)
len(ml_sequence)

25

In [75]:
# Let's see x first tokenized words of specified article index

ml_article_idx = 1
ml_x = 10

ml_sequence[ml_article_idx][:ml_x]

[132, 865, 582, 583, 866, 224, 33, 132, 867, 868]

In [76]:
print('In Money Laundering train dataset we have {} unique words'.format(len(ml_tokenizer.word_index)))

In Money Laundering train dataset we have 3487 unique words


## Pad_sequences

In [77]:
# Pad sequences - each article has different number of words/sequences to make it even for all articles we will use pad_sequences method reducing/extending each up to 1000 words

ml_train_txt = pad_sequences(ml_sequence, maxlen = max_len)
print(ml_train_txt)
print('\n')
print(ml_train_txt.shape)

[[  0   0   0 ... 846 581  61]
 [  0   0   0 ... 411 298 590]
 [424 613 425 ... 894 413 309]
 ...
 [  0   0   0 ... 482 124 204]
 [  0   0   0 ...  69  56 836]
 [  1 477 773 ... 572   8 368]]


(25, 200)


## Shuffeling

In [78]:
# convert class list into an array for sake of shuffeling
ml_train_class = np.asarray(ml_class)
ml_train_class

array([1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 0])

In [79]:
ml_idx = np.arange(ml_train_txt.shape[0]) # create indexes array of ml_train
np.random.shuffle(ml_idx) # shuffle
ml_train_txt = ml_train_txt[ml_idx] # change order in texts
ml_train_class = ml_train_class[ml_idx] # change order in class' in the same way
print(ml_train_txt.shape)
print(ml_train_class.shape)

(25, 200)
(25,)


## Train Validation split

In [80]:
ml_train_percent = 0.6
ml_validation_percent = 0.2

ml_separator = int(ml_train_percent * (ml_train_txt.shape[0]))
ml_separator2 = int(ml_train_txt.shape[0] * ml_validation_percent) + ml_separator

ml_X_train = ml_train_txt[:ml_separator]
ml_y_train = ml_train_class[:ml_separator]

ml_X_val = ml_train_txt[ml_separator:ml_separator2]
ml_y_val = ml_train_class[ml_separator:ml_separator2]

ml_X_test = ml_train_txt[ml_separator2:]
ml_y_test = ml_train_class[ml_separator2:]

print('X_train shape:',ml_X_train.shape)
print('y_train shape:',ml_y_train.shape)
print('X_val shape:',ml_X_val.shape)
print('y_val shape:',ml_y_val.shape)
print('X_test shape:',ml_X_test.shape)
print('y_test shape:',ml_y_test.shape)

X_train shape: (15, 200)
y_train shape: (15,)
X_val shape: (5, 200)
y_val shape: (5,)
X_test shape: (5, 200)
y_test shape: (5,)


## Model

### Model hyperparameters

In [81]:
optimizer = 'rmsprop'
batch_size = 8
epochs = 11

In [82]:
ml_model = Sequential([
                    Embedding(words_cnt, target_dim, input_length = max_len),
                    Bidirectional(LSTM(100)),
                    Dense(16, activation = 'relu'),
                    Dense(1, activation = 'sigmoid')
])
ml_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 200, 10)           10000     
_________________________________________________________________
bidirectional_4 (Bidirection (None, 200)               88800     
_________________________________________________________________
dense_12 (Dense)             (None, 16)                3216      
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 17        
Total params: 102,033
Trainable params: 102,033
Non-trainable params: 0
_________________________________________________________________


In [83]:
ml_model.compile(
              optimizer = optimizer,
              loss = 'binary_crossentropy',
              metrics = ['accuracy']
)

In [84]:
ml_history = ml_model.fit(ml_X_train, ml_y_train, batch_size = batch_size, epochs = epochs, validation_data = (ml_X_val, ml_y_val))

Epoch 1/11
2/2 [==============================] - 4s 932ms/step - loss: 0.6953 - accuracy: 0.3333 - val_loss: 0.6973 - val_accuracy: 0.4000
Epoch 2/11
2/2 [==============================] - 0s 214ms/step - loss: 0.6882 - accuracy: 0.6000 - val_loss: 0.7010 - val_accuracy: 0.4000
Epoch 3/11
2/2 [==============================] - 0s 202ms/step - loss: 0.6781 - accuracy: 0.6000 - val_loss: 0.7082 - val_accuracy: 0.4000
Epoch 4/11
2/2 [==============================] - 0s 217ms/step - loss: 0.7991 - accuracy: 0.6000 - val_loss: 0.7053 - val_accuracy: 0.4000
Epoch 5/11
2/2 [==============================] - 0s 204ms/step - loss: 0.6682 - accuracy: 0.6000 - val_loss: 0.7055 - val_accuracy: 0.4000
Epoch 6/11
2/2 [==============================] - 0s 211ms/step - loss: 0.6574 - accuracy: 0.6000 - val_loss: 0.7072 - val_accuracy: 0.4000
Epoch 7/11
2/2 [==============================] - 0s 202ms/step - loss: 0.6513 - accuracy: 0.6000 - val_loss: 0.7076 - val_accuracy: 0.4000
Epoch 8/11
2/2 [====

## Train-Val Plot

In [85]:
def plot_history(history):
  import pandas as pd
  import plotly.graph_objects as go
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  fig = go.Figure()
  fig.add_trace(go.Scatter(x = hist['epoch'], y = hist['accuracy'], name = 'accuracy', mode = 'markers+lines'))
  fig.add_trace(go.Scatter(x = hist['epoch'], y = hist['val_accuracy'], name = 'val_accuracy', mode = 'markers+lines'))
  fig.update_layout(width = 1000, height = 500, title = 'accuracy vs val. accuracy', xaxis_title = 'Epochs', yaxis_title = 'Accuracy', template = 'plotly_dark')
  fig.show()

  fig = go.Figure()
  fig.add_trace(go.Scatter(x = hist['epoch'], y = hist['loss'], name = 'loss', mode = 'markers+lines'))
  fig.add_trace(go.Scatter(x = hist['epoch'], y = hist['val_loss'], name = 'val_loss', mode = 'markers+lines'))
  fig.update_layout(width = 1000, height = 500, title = 'loss vs val. loss', xaxis_title = 'Epochs', yaxis_title = 'Loss', template = 'plotly_dark')
  fig.show()

plot_history(ml_history)

## Evaluate

In [86]:
ml_evaluate = ml_model.evaluate(ml_X_test, ml_y_test, verbose = 0)

print('Model loss on test set:',round(ml_evaluate[0],2))
print('Model accuracy on test set:',round(ml_evaluate[1],2))

Model loss on test set: 0.73
Model accuracy on test set: 0.2


As we can see model didn't perform well. Obviously it is overfitting. For better performance we need better data set assembled from more articles.

## Save model

In [87]:
ml_model.save('ml_model')

INFO:tensorflow:Assets written to: ml_model/assets


INFO:tensorflow:Assets written to: ml_model/assets


# Allegation and Conviction detection



---



---



## Full code

In [88]:
### STOPWORDS ###
for i in range(len(ind_txt)):
  ind_txt[i] = remove_stopwords(ind_txt[i])

### TOKENIZING ###
ind_tokenizer = Tokenizer(num_words = words_cnt)
ind_tokenizer.fit_on_texts(ind_txt)

### SEQUENCING ###
ind_sequence = ind_tokenizer.texts_to_sequences(ind_txt)

### PAD SEQUENCES ###
ind_train_txt = pad_sequences(ind_sequence, maxlen = max_len)

### SHUFFELING ###
ind_train_class = np.asarray(ind_class)

ind_idx = np.arange(ind_train_txt.shape[0]) # create indexes array of ml_train
np.random.shuffle(ind_idx) # shuffle
ind_train_txt = ind_train_txt[ind_idx] # change order in texts
ind_train_class = ind_train_class[ind_idx] 

### TRAIN-VAL-TEST SPLIT ###
ind_train_percent = 0.6
ind_validation_percent = 0.2

ind_separator = int(ind_train_percent * (ind_train_txt.shape[0]))
ind_separator2 = int(ind_train_txt.shape[0] * ind_validation_percent) + ind_separator

ind_X_train = ind_train_txt[:ind_separator]
ind_y_train = ind_train_class[:ind_separator]

ind_X_val = ind_train_txt[ind_separator:ind_separator2]
ind_y_val = ind_train_class[ind_separator:ind_separator2]

ind_X_test = ind_train_txt[ind_separator2:]
ind_y_test = ind_train_class[ind_separator2:]

### MODEL ###
ind_model = Sequential([
                    Embedding(words_cnt, target_dim, input_length = max_len),
                    Bidirectional(LSTM(100)),
                    Dense(32, activation = 'selu'),
                    Dense(32, activation = 'relu'),
                    Dense(16),
                    Dense(1, activation = 'sigmoid')
])

ind_model.compile(
              optimizer = optimizer,
              loss = 'binary_crossentropy',
              metrics = ['accuracy']
)

ind_history = ind_model.fit(ind_X_train, ind_y_train, batch_size = batch_size, epochs = epochs, validation_data = (ind_X_val, ind_y_val))
print('\n\n')

### TRAIN-VAL PLOT ###
plot_history(ind_history)
print('\n\n')

### EVALUATE ###
ind_evaluate = ind_model.evaluate(ind_X_test, ind_y_test, verbose = 0)

print('Model loss on test set:',round(ind_evaluate[0],2))
print('Model accuracy on test set:',round(ind_evaluate[1],2))

## SAVE MODEL ###
ind_model.save('ind_model')

Epoch 1/11
2/2 [==============================] - 5s 970ms/step - loss: 0.6915 - accuracy: 0.7500 - val_loss: 0.6869 - val_accuracy: 0.5000
Epoch 2/11
2/2 [==============================] - 0s 191ms/step - loss: 0.6715 - accuracy: 0.6667 - val_loss: 0.6830 - val_accuracy: 0.5000
Epoch 3/11
2/2 [==============================] - 0s 188ms/step - loss: 0.6314 - accuracy: 0.6667 - val_loss: 0.6576 - val_accuracy: 0.5000
Epoch 4/11
2/2 [==============================] - 0s 190ms/step - loss: 1.0492 - accuracy: 0.8333 - val_loss: 0.6549 - val_accuracy: 0.5000
Epoch 5/11
2/2 [==============================] - 0s 199ms/step - loss: 0.5585 - accuracy: 1.0000 - val_loss: 0.6401 - val_accuracy: 0.5000
Epoch 6/11
2/2 [==============================] - 0s 196ms/step - loss: 0.4769 - accuracy: 1.0000 - val_loss: 0.8009 - val_accuracy: 0.5000
Epoch 7/11
2/2 [==============================] - 0s 190ms/step - loss: 0.4377 - accuracy: 0.8333 - val_loss: 0.6143 - val_accuracy: 0.5000
Epoch 8/11
2/2 [====




Model loss on test set: 0.68
Model accuracy on test set: 0.5


INFO:tensorflow:Assets written to: ind_model/assets


INFO:tensorflow:Assets written to: ind_model/assets


# Running both models



---



---



## Application

In [89]:
class Detector:
  def __init__(self):
    # --------------------- Libraries------------------------ #

    from google.colab import files
    from tensorflow import keras
    import pandas as pd
    import numpy as np

    # --------------------- Import datasets ------------------------ #

    # print('Select: \n1.articles dataframe\n2.money laundering model \n3.indictiment model\n')
    # files.upload()

    articles = pd.read_excel('articles.xlsx')
    ml_model = keras.models.load_model('ml_model')
    ind_model = keras.models.load_model('ind_model')

    self.articles = articles
    self.ml_model = ml_model
    self.ind_model = ind_model

    # --------------------- Display input ------------------------ #

  # SHOW ALL ARTICLES #
  def articles(self):
    return self.articles

  # SHOW ML_MODEL SUMMARY #
  def ml_model(self):
    return self.ml_model.summary()

  # SHOW IND_MODEL SUMMARY #
  def ind_model(self):
    return self.ind_model.summary()

    # --------------------- Money Laundering or Not Detection ------------------------ #

  # PREPROCESS DATA TO ML DETECTION #
  def ml_preprocessing(self):

    ### CREATING A LIST OF ARTICLES ###
    txt = []
    for i in self.articles['text']:
      txt.append(i)
    ### SETTING UP HYPERPARAMETERS ###
    words_cnt = 1000
    max_len = 200

    ### TOKENIZING ###
    tokenizer = Tokenizer(num_words = words_cnt)
    tokenizer.fit_on_texts(txt)
    ### SEQUENCING ###
    sequence = tokenizer.texts_to_sequences(txt)
    ### PAD SEQUENCES ###
    padd_txt = pad_sequences(sequence, maxlen = max_len)

    return padd_txt

  # SHOW DF WITH ALL ARTICLES AND 'MONEY_LAUNDERING_OR_NOT' BINAR COLUMN #
  def ml_predict(self):
    # articles = self.articles
    ml_padd_txt = self.ml_preprocessing()

    ml_predictions = [1 if p > 0.5 else 0 for p in self.ml_model.predict(ml_padd_txt)]

    ml_articles = self.articles.copy()
    ml_articles['money_laundering_or_not'] = ml_predictions

    return ml_articles

  # SHOW SIGMOID FUNCTION VALUES OF ML DETECTION FOR EACH ARTICLE #
  def ml_sigmoid_values(self):
    ml_padd_txt = self.ml_preprocessing()

    ml_sigmoid_values = self.ml_model.predict(ml_padd_txt)

    return ml_sigmoid_values

  # RETURN DF WITH ARTICLES THAT WERE DETECTED AS MONEY LAUNDERING #
  def ml_filtered(self):
    ml_articles = self.ml_predict()

    if len(ml_articles) == 0:
      return print('No articles detected as money laundering')
    else:
      filtered_ml_articles = ml_articles[ml_articles['money_laundering_or_not'] == 1]

    return filtered_ml_articles

  # --------------------- Allegation or Conviction Detection ------------------------ #

  # PREPROCESS FILTERED DF TO ALLEGATION OR CONVICTION DETECTION #
  def ind_preprocessing(self):
    ml_articles = self.ml_predict()

    if len(ml_articles) == 0:
      return print('No articles detected as money laundering')
    else:
      ind_articles = self.ml_filtered()
      ### CREATING A LIST OF ARTICLES ###
      ind_txt = []
      for i in ind_articles['text']:
        ind_txt.append(i)
      ### SETTING UP HYPERPARAMETERS ###
      ind_words_cnt = 1000
      ind_max_len = 100
      ### TOKENIZING ###
      ind_tokenizer = Tokenizer(num_words = ind_words_cnt)
      ind_tokenizer.fit_on_texts(ind_txt)
      ### SEQUENCING ###
      ind_sequence = ind_tokenizer.texts_to_sequences(ind_txt)
      ### PAD SEQUENCES ###
      ind_padd_txt = pad_sequences(ind_sequence, maxlen = ind_max_len)
      return ind_padd_txt

  # SHOW LIST OF PREDICTIONS OF ALLEGATION OR CONVICTION DETECTION #
  def ind_predictions(self):
    ind_padd_txt = self.ind_preprocessing()
    ml_articles = self.ml_predict()
      
    if len(ml_articles) == 0:

      return print('No articles detected as money laundering')
    else:
      ind_predictions = [1 if p > 0.5 else 0 for p in ind_model.predict(ind_padd_txt)]

      return ind_predictions

  # RETURN DF WITH PREDICTIONS FOR ALLEGATION OR CONVICTION DETECTION #
  def ind_predict(self):
    ml_articles = self.ml_predict()

    if len(ml_articles) == 0:

      return print('No articles detected as money laundering')
    else:
      ind_predictions = self.ind_predictions()
      ind_articles = self.ml_filtered()

      ind_articles['conviction'] = ind_predictions
      ind_articles.drop('money_laundering_or_not', axis = 1, inplace = True)

      return ind_articles

  # SHOW ARRAY OF SIGMOID FUNCTION VALUES OF ML DETECTION FOR EACH ARTICLE #
  def ind_sigmoid_values(self):
    ind_padd_txt = self.ind_preprocessing()

    ind_sigmoid_values = self.ind_model.predict(ind_padd_txt)

    return ind_sigmoid_values

  # SHOW DF WITH FULL ARTICLES LIST AND BOTH PREDICTIONS (MONEY LAUNDERING OR NOT AND ALLEGTION OR CONVICTION DETECTION)
  def predict(self):
    ml_articles = self.ml_predict()
    ind_articles = self.ind_predict()

    if len(ml_articles) == 0:
        return print('No articles detected as money laundering')
    else:
        all_articles = ml_articles.merge(ind_articles, on='text', how = 'left').fillna(0)
        all_articles['conviction'] = all_articles['conviction'].apply(lambda x: int(x))

        return all_articles

## Execution

In [90]:
detector = Detector()

Sigmoid values for Money Laundering articles detection

In [91]:
detector.ml_sigmoid_values()

array([[0.5126577 ],
       [0.50596124],
       [0.5090526 ],
       [0.51065964],
       [0.5127673 ],
       [0.5122412 ],
       [0.4964424 ],
       [0.5093552 ],
       [0.51248527],
       [0.5034157 ],
       [0.51381105],
       [0.52324027],
       [0.515902  ],
       [0.51062375]], dtype=float32)

Sigmoid values for Allegation or Conviction articles detection

In [92]:
detector.ind_sigmoid_values()

array([[0.540378  ],
       [0.6493387 ],
       [0.4745122 ],
       [0.56895477],
       [0.5922787 ],
       [0.507244  ],
       [0.5494748 ],
       [0.6948388 ],
       [0.4971142 ],
       [0.58169407],
       [0.66764104],
       [0.53893757],
       [0.39143068]], dtype=float32)

DataFrame of full predictions for each article

In [93]:
detector.predict()

,text,money_laundering_or_not,conviction
0,"Allegations of money laundering, fraud, crimin...",1,1
1,Three Men Charged with Laundering Proceeds of ...,1,1
2,"Son of ex-Pension boss, Faisal Maina, bags 14y...",1,0
3,Ex-adviser jailed after laundering money for o...,1,1
4,Oregon Court of Appeals overturns conviction o...,1,1
5,MEPs ‘strongly condemn’ conviction of ‘Hotel R...,1,1
6,he Black Film Archive wants to show the world ...,0,0
7,Jury convicts ex-Northwestern professor of mur...,1,1
8,Sensory Design: Architecture for a Full Spectr...,1,1
9,UAE central bank issues new anti-money launder...,1,0


DataFrame of Money Laundering predictions for each article

In [94]:
detector.ml_predict()

,text,money_laundering_or_not
0,"Allegations of money laundering, fraud, crimin...",1
1,Three Men Charged with Laundering Proceeds of ...,1
2,"Son of ex-Pension boss, Faisal Maina, bags 14y...",1
3,Ex-adviser jailed after laundering money for o...,1
4,Oregon Court of Appeals overturns conviction o...,1
5,MEPs ‘strongly condemn’ conviction of ‘Hotel R...,1
6,he Black Film Archive wants to show the world ...,0
7,Jury convicts ex-Northwestern professor of mur...,1
8,Sensory Design: Architecture for a Full Spectr...,1
9,UAE central bank issues new anti-money launder...,1


DataFrame of Allegation or Conviction prediction for each article

In [95]:
detector.ind_predict()

,text,conviction
0,"Allegations of money laundering, fraud, crimin...",1
1,Three Men Charged with Laundering Proceeds of ...,1
2,"Son of ex-Pension boss, Faisal Maina, bags 14y...",0
3,Ex-adviser jailed after laundering money for o...,1
4,Oregon Court of Appeals overturns conviction o...,1
5,MEPs ‘strongly condemn’ conviction of ‘Hotel R...,1
7,Jury convicts ex-Northwestern professor of mur...,1
8,Sensory Design: Architecture for a Full Spectr...,1
9,UAE central bank issues new anti-money launder...,0
10,The mystery of Elon Musk’s missing gas\nSpaceX...,1




---



---

